In [1]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

In [2]:
pip install keras-preprocessing

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for keras-preprocessing from https://files.pythonhosted.org/packages/79/4c/7c3275a01e12ef9368a892926ab932b33bb13d55794881e3573482b378a7/Keras_Preprocessing-1.1.2-py2.py3-none-any.whl.metadata
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl.metadata (1.9 kB)
Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)


In [2]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

In [24]:
TRAIN_DIR = 'images/train'
TEST_DIR = 'images/test'

In [25]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths,labels

In [26]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

Angry completed
Disgust completed
Fear completed
Happy completed
Neutral completed
Sad completed
Surprise completed


In [27]:
print(train)

                                 image     label
0          images/train\Angry\1337.jpg     Angry
1          images/train\Angry\1338.jpg     Angry
2          images/train\Angry\1339.jpg     Angry
3          images/train\Angry\1340.jpg     Angry
4          images/train\Angry\1341.jpg     Angry
...                                ...       ...
27293  images/train\Surprise\12551.jpg  Surprise
27294  images/train\Surprise\12552.jpg  Surprise
27295  images/train\Surprise\12553.jpg  Surprise
27296  images/train\Surprise\12554.jpg  Surprise
27297  images/train\Surprise\12555.jpg  Surprise

[27298 rows x 2 columns]


In [28]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

Angry completed
Disgust completed
Fear completed
Happy completed
Neutral completed
Sad completed
Surprise completed


In [29]:
print(test['image'])

0            images/test\Angry\1337.jpg
1            images/test\Angry\1338.jpg
2            images/test\Angry\1339.jpg
3            images/test\Angry\1340.jpg
4            images/test\Angry\1341.jpg
                      ...              
27298    images/test\Surprise\12551.jpg
27299    images/test\Surprise\12552.jpg
27300    images/test\Surprise\12553.jpg
27301    images/test\Surprise\12554.jpg
27302    images/test\Surprise\12555.jpg
Name: image, Length: 27303, dtype: object


In [30]:
from tqdm.notebook import tqdm

In [31]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,grayscale =  True )
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),112,112,1)
    return features
    

In [32]:
train_features = extract_features(train['image']) 

  0%|          | 0/27298 [00:00<?, ?it/s]

C:\Users\CG Lab\Desktop\project\face\Lib\site-packages\keras_preprocessing\image\utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [33]:
test_features = extract_features(test['image'])

  0%|          | 0/27303 [00:00<?, ?it/s]

In [13]:
x_train = train_features/255.0
x_test = test_features/255.0

In [14]:
from sklearn.preprocessing import LabelEncoder

In [15]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [16]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [17]:
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)

In [20]:
model = Sequential()
# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(7, activation='softmax'))

C:\Users\CG Lab\Desktop\project\face\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy')

In [22]:
model.fit(x= x_train,y = y_train, batch_size = 128, epochs = 50, validation_data = (x_test,y_test)) 

Epoch 1/50
226/226 ━━━━━━━━━━━━━━━━━━━━ 93s 400ms/step - loss: 1.8365 - val_loss: 1.8138
Epoch 2/50
226/226 ━━━━━━━━━━━━━━━━━━━━ 142s 398ms/step - loss: 1.8097 - val_loss: 1.7810
Epoch 3/50
226/226 ━━━━━━━━━━━━━━━━━━━━ 92s 408ms/step - loss: 1.7741 - val_loss: 1.6722
Epoch 4/50
226/226 ━━━━━━━━━━━━━━━━━━━━ 92s 409ms/step - loss: 1.6758 - val_loss: 1.5560
Epoch 5/50
226/226 ━━━━━━━━━━━━━━━━━━━━ 93s 411ms/step - loss: 1.5524 - val_loss: 1.3892
Epoch 6/50
226/226 ━━━━━━━━━━━━━━━━━━━━ 93s 409ms/step - loss: 1.4694 - val_loss: 1.3381
Epoch 7/50
226/226 ━━━━━━━━━━━━━━━━━━━━ 140s 402ms/step - loss: 1.4130 - val_loss: 1.3089
Epoch 8/50
226/226 ━━━━━━━━━━━━━━━━━━━━ 93s 410ms/step - loss: 1.3749 - val_loss: 1.2578
Epoch 9/50
226/226 ━━━━━━━━━━━━━━━━━━━━ 93s 410ms/step - loss: 1.3486 - val_loss: 1.2208
Epoch 10/50
226/226 ━━━━━━━━━━━━━━━━━━━━ 93s 411ms/step - loss: 1.3268 - val_loss: 1.2083
Epoch 11/50
226/226 ━━━━━━━━━━━━━━━━━━━━ 93s 413ms/step - loss: 1.3069 - val_loss: 1.1967
Epoch 12/50
226/2

In [23]:
model_json = model.to_json()
with open("emotiondetector.json",'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")